In [15]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, classification_report
from transformers import T5ForConditionalGeneration, AutoTokenizer
import joblib

In [2]:
ds = load_dataset("PolyAI/minds14", "all")

In [3]:
train_df = ds['train'].to_pandas()[['english_transcription', 'intent_class']]
train_df

,english_transcription,intent_class
0,"Hello, I need to find out what is the maximum ...",3
1,Hello I will withdraw money from your ATM for ...,3
2,What is the limit I can choose from an ATM,3
3,How much money can I withdraw from an ATM,3
4,"Hello, I would like to ask you I would like to...",3
...,...,...
8163,Can you help me get my card? My card can’t be ...,6
8164,The card is invalid and the payment is declined,6
8165,Card cannot be used and cannot be paid,6
8166,"Hello, I just used my bank card to make a paym...",6


In [4]:
train_df["intent_class"].value_counts()


intent_class
0     654
1     627
7     608
6     601
4     598
3     596
11    596
5     580
13    577
9     562
12    562
8     551
10    549
2     507
Name: count, dtype: int64

In [5]:
# Classlar dengeli duruyor. Her bir class değeri şunları temsil etmektedir.
"""
0 -> abroad
1 -> address
2 -> app_error
3 -> atm_limit
4 -> balance
5 -> business_loan
6 -> card_issues
7 -> cash_deposit
8 -> direct_debit
9 -> freeze
10 -> high_value_payment
11 -> joint_account
12 -> latest_transactions
13 -> pay_bill
"""

'\n0 -> abroad\n1 -> address\n2 -> app_error\n3 -> atm_limit\n4 -> balance\n5 -> business_loan\n6 -> card_issues\n7 -> cash_deposit\n8 -> direct_debit\n9 -> freeze\n10 -> high_value_payment\n11 -> joint_account\n12 -> latest_transactions\n13 -> pay_bill\n'

In [6]:
# Sınıfı çektik. Sınıfı embed edip ML modeline vereceğiz. İyi sonuç alırsak o modeli kullanacağız.
# Gelen metni düzeltip o modele vereceğiz.
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(text:str):
    embeddings = model.encode([text], show_progress_bar=True, normalize_embeddings=True)
    return embeddings[0]

train_df["embeddings"] = train_df["english_transcription"].apply(embed_text)
train_df

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,english_transcription,intent_class,embeddings
0,"Hello, I need to find out what is the maximum ...",3,"[0.08434033, -0.00047242446, -0.06478009, -0.0..."
1,Hello I will withdraw money from your ATM for ...,3,"[-0.008761657, 0.022830687, -0.04328943, 0.026..."
2,What is the limit I can choose from an ATM,3,"[0.038601704, -0.0074460264, -0.07636333, -0.0..."
3,How much money can I withdraw from an ATM,3,"[0.10925046, 0.054237634, -0.07468217, -0.0317..."
4,"Hello, I would like to ask you I would like to...",3,"[0.08008452, 0.01159375, -0.062252928, -0.0599..."
...,...,...,...
8163,Can you help me get my card? My card can’t be ...,6,"[-0.012982257, 0.0031667775, 0.02079224, 0.014..."
8164,The card is invalid and the payment is declined,6,"[-0.0047067045, 0.062449723, 0.016953183, -0.0..."
8165,Card cannot be used and cannot be paid,6,"[-0.04758662, 0.061233334, -0.017234279, 0.005..."
8166,"Hello, I just used my bank card to make a paym...",6,"[0.06808036, -0.0066186213, 0.021433245, 0.045..."


In [7]:
train_df

,english_transcription,intent_class,embeddings
0,"Hello, I need to find out what is the maximum ...",3,"[0.08434033, -0.00047242446, -0.06478009, -0.0..."
1,Hello I will withdraw money from your ATM for ...,3,"[-0.008761657, 0.022830687, -0.04328943, 0.026..."
2,What is the limit I can choose from an ATM,3,"[0.038601704, -0.0074460264, -0.07636333, -0.0..."
3,How much money can I withdraw from an ATM,3,"[0.10925046, 0.054237634, -0.07468217, -0.0317..."
4,"Hello, I would like to ask you I would like to...",3,"[0.08008452, 0.01159375, -0.062252928, -0.0599..."
...,...,...,...
8163,Can you help me get my card? My card can’t be ...,6,"[-0.012982257, 0.0031667775, 0.02079224, 0.014..."
8164,The card is invalid and the payment is declined,6,"[-0.0047067045, 0.062449723, 0.016953183, -0.0..."
8165,Card cannot be used and cannot be paid,6,"[-0.04758662, 0.061233334, -0.017234279, 0.005..."
8166,"Hello, I just used my bank card to make a paym...",6,"[0.06808036, -0.0066186213, 0.021433245, 0.045..."


In [8]:
X = train_df["embeddings"].to_list()
y = train_df["intent_class"].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)
# DBScan ve KMeans KNN kadar iyi sonuç vermedi. Embeddingleri normalize edip uzaya göre gruplamak mantıklı geldiği için cluster algoritmalarını denedim.
knn_model = KNeighborsClassifier(n_neighbors=14, metric='cosine')
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       131
           1       0.99      1.00      1.00       126
           2       0.98      0.97      0.98       101
           3       0.97      0.98      0.98       119
           4       0.94      0.97      0.96       120
           5       0.97      0.98      0.98       116
           6       0.92      1.00      0.96       120
           7       0.94      0.96      0.95       122
           8       0.97      0.89      0.93       110
           9       1.00      0.93      0.96       112
          10       0.98      0.94      0.96       110
          11       0.99      0.97      0.98       119
          12       0.91      0.94      0.92       112
          13       0.95      0.97      0.96       116

    accuracy                           0.96      1634
   macro avg       0.96      0.96      0.96      1634
weighted avg       0.96      0.96      0.96      1634

F1 Score: 0.9632598295947

In [9]:
correction_model = T5ForConditionalGeneration.from_pretrained("ai-forever/T5-large-spell")
tokenizer = AutoTokenizer.from_pretrained("ai-forever/T5-large-spell")
# prefix olarak grammar: ya da correct spelling: ekleyebiliriz. Test örneğinde correct spelling: in daha kullanışlı olduğunu gördüm.
prefix = "grammar: "
sentence = "This is a smaple sentnce with erors."
input_text = prefix + sentence
encodings = tokenizer(input_text, return_tensors="pt")
generated_tokens = correction_model.generate(**encodings)
answer = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print("Corrected Sentence:", answer)

Corrected Sentence: ['This is a simple sentence with errors.']


In [10]:
# Veri setinde olmayan elle oluşturulan test verimiz
test_dataset = {
    "im travelling abrod next wek and not sure if my card wil work outsid the contry": 0,
    "i recenty moved to a new apartmant and need to updat my home adress in the system": 1,
    "im geting an unexpcted eror mesage when i try to make a tranfer thru the app": 2,
    "whats my max cash withdrawl limit from atm s": 3,
    "how much is left in my cheking acount after my recent transacions": 4,
    "im planing to expand my compny and would like info about bussines loans": 5,
    "my card was declind at the store even tho i have enogh funds": 6,
    "is it posible to make a cash deposet thru an atm": 7,
    "i want to setup otomatik payments for my monthy bills": 8,
    "how do i tempory block my card for securty reasons": 9,
    "what verifcation is requierd for large transfars": 10,
    "i want to add anothr person to my exsisting bank acount": 11,
    "can u show me the lastest activty on my acount": 12,
    "i need to pay my electricty bill usin my bank acount": 13}

In [11]:
correction_model = T5ForConditionalGeneration.from_pretrained("ai-forever/T5-large-spell")
tokenizer = AutoTokenizer.from_pretrained("ai-forever/T5-large-spell")
prefix = "correct spelling: "
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

y_preds = []
y_trues = list(test_dataset.values())

for sentence, y in test_dataset.items():
    input_text = prefix + sentence
    encodings = tokenizer(input_text, return_tensors="pt")
    generated_tokens = correction_model.generate(**encodings)
    corr = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(corr)
    embeddings = embed_model.encode(corr, show_progress_bar=True, normalize_embeddings=True)
    pred = knn_model.predict(embeddings)
    y_preds.append(pred[0])

# Accuracy i manual hesaplayalım
correct_count = sum(1 for true, pred in zip(y_trues, y_preds) if true == pred) 
print("Accuracy:", correct_count / len(y_trues)) 


['I am travelling abroad next week and not sure if my card will work outside the country']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['I recently moved to a new apartment and need to update my home address in the system']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['im getting an unexpected error message when i try to make a transfer thru the app']


...

['can u show me the latest activity on my account']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['i need to pay my electricity bill using my bank account']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 0.9285714285714286


In [12]:
y_trues, y_preds

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 [np.int64(0),
  np.int64(1),
  np.int64(2),
  np.int64(3),
  np.int64(4),
  np.int64(5),
  np.int64(6),
  np.int64(7),
  np.int64(13),
  np.int64(9),
  np.int64(10),
  np.int64(11),
  np.int64(12),
  np.int64(13)])

In [16]:
joblib.dump(knn_model, "knn_intent_model.joblib")


['knn_intent_model.joblib']